In [5]:
import os
import warnings
from openai import OpenAI

# Define OpenAI API_KEY
with open("/home/savitha07/.env") as env:
    for line in env:
        key, value = line.strip().split('=')
        os.environ[key] = value

client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY'),
)

os.environ["TAVILY_API_KEY"] = os.environ.get('OPENAI_API_KEY')

warnings.filterwarnings("ignore")


In [6]:
import datetime

current_date = datetime.datetime.now().date()
target_date = datetime.date(2024, 6, 12)

if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [7]:
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(llm_model)

In [13]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

prompt = hub.pull("hwchase17/openai-functions-agent")

print(llm_model)

llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What is the 25% of 300?"})


gpt-3.5-turbo-0301


> Entering new AgentExecutor chain...
To find 25% of 300, you can multiply 300 by 0.25. 

25% of 300 = 300 * 0.25 = 75

Therefore, 25% of 300 is 75.

> Finished chain.


{'input': 'What is the 25% of 300?',
 'output': 'To find 25% of 300, you can multiply 300 by 0.25. \n\n25% of 300 = 300 * 0.25 = 75\n\nTherefore, 25% of 300 is 75.'}

In [14]:
agent_executor.invoke({"input": "What is Langchain?"})




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Langchain'}`


HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')I'm sorry, but I couldn't retrieve information about Langchain at the moment. Would you like me to try searching for information on a different platform or provide information based on what I know?

> Finished chain.


{'input': 'What is Langchain?',
 'output': "I'm sorry, but I couldn't retrieve information about Langchain at the moment. Would you like me to try searching for information on a different platform or provide information based on what I know?"}

In [15]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Don't use tools to look this up unless you NEED to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you further, Bob?

> Finished chain.


{'input': "what's my name? Don't use tools to look this up unless you NEED to",
 'chat_history': [HumanMessage(content='hi! my name is bob'),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob. How can I assist you further, Bob?'}

In [20]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


In [24]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import tool
from datetime import date

python_repl = PythonREPL()

repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wiki_tool = Tool(
    name="wikipedia",
    description="A tool for querying Wikipedia to retrieve information on a specific topic. Use this tool to perform searches and gather relevant details from the vast knowledge available on Wikipedia.",
    func=wikipedia.run,
)


@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

tools = [repl_tool, wiki_tool, time]
prompt = hub.pull("hwchase17/openai-functions-agent")

print(llm_model)

llm = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0)

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



gpt-3.5-turbo-0301


In [25]:
agent_executor.invoke({"input": "What is the 25% of 300?"})



> Entering new AgentExecutor chain...

Invoking: `python_repl` with `300 * 0.25`
responded: To find 25% of 300, you can multiply 300 by 0.25. 

Let's calculate it:

25% of 300 is 75.

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '25% of 300 is 75.'}

In [26]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie \
Mellon University (CMU)\
what book did he write?"

agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Tom M. Mitchell`


Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Oren Etzioni
Summary: Oren Etzioni (born 1964) 

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Tom M. Mitchell wrote the textbook "Machine Learning".'}

In [27]:
agent_executor.invoke({"input": "What's the date today?"})



> Entering new AgentExecutor chain...

Invoking: `time` with `{'text': ''}`


2024-03-05Today is March 5, 2024. How can I assist you further?

> Finished chain.


{'input': "What's the date today?",
 'output': 'Today is March 5, 2024. How can I assist you further?'}

In [28]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

question = f"Sort these customers by \
last name and then first name \
and print the output: {customer_list}"

agent_executor.invoke({"input": question})



> Entering new AgentExecutor chain...

Invoking: `python_repl` with `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]

sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))

sorted_customers`


Here is the sorted list of customers by last name and then first name:

[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Dolly', 'Too'], ['Trance', 'Former']]

> Finished chain.


{'input': "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
 'output': "Here is the sorted list of customers by last name and then first name:\n\n[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Dolly', 'Too'], ['Trance', 'Former']]"}